# JOIN

In [17]:
import sqlite3

In [18]:
# sqlite와 connection 만들기.
#   파일에저장해서 DBbrowser에서 확인하던지, 메모리에 잠깐 저장하던지.
conn = sqlite3.connect("test.db")

In [19]:
cur = conn.cursor()

In [31]:
cur.executescript(
    """
        drop table artist;
        drop table genre;
        drop table album;
        drop table track;
    """
);

In [32]:
cur.executescript(
    """
        create table artist (
            id integer primary key autoincrement,
            name text
        );
        create table genre (
            id integer primary key autoincrement,
            name text
        );
        create table album (
            id integer primary key autoincrement,
            title text,
            artist_id integer
        );
        create table track (
            id integer primary key autoincrement,
            title text,
            length integer,
            count integer,
            rating integer,
            album_id integer,
            genre_id integer
        );
    """
)
# album테이블의 artist_id에 대해서 따로 FK라고 선언 안함.

In [33]:
conn.commit();

In [34]:
cur.executescript(
    """
        insert into artist (name) values('Led Zepplin');
        insert into artist (name) values('AC/DC');
        
        insert into genre (name) values('Rock');
        insert into genre (name) values('Metal');
    """
)

In [35]:
cur.execute("select * from artist")

In [36]:
print(cur.fetchall())

[(1, 'Led Zepplin'), (2, 'AC/DC')]


In [37]:
artist = cur.fetchall()
type(artist)

list

In [38]:
cur.executescript(
    """
        insert into album (title, artist_id) values('IV', 1);
        insert into album (title, artist_id) values('Who Made Who', 2);
        
        insert into track (title, album_id, genre_id) values('Black Dog', 1, 1);
        insert into track (title, album_id, genre_id) values('Stairway', 1, 2);
        insert into track (title, album_id, genre_id) values('About to Rock', 2, 1);
        insert into track (title, album_id, genre_id) values('Who Made Who', 2, 2);
    """
);

In [41]:
cur.execute("select * from track")

In [43]:
print(cur.fetchall())

[(1, 'Black Dog', None, None, None, 1, 1), (2, 'Stairway', None, None, None, 1, 2), (3, 'About to Rock', None, None, None, 2, 1), (4, 'Who Made Who', None, None, None, 2, 2)]


In [44]:
# Cross Join.

cur.execute("select * from artist cross join album")

In [45]:
for row in cur.fetchall():
    print(row)

(1, 'Led Zepplin', 1, 'IV', 1)
(1, 'Led Zepplin', 2, 'Who Made Who', 2)
(2, 'AC/DC', 1, 'IV', 1)
(2, 'AC/DC', 2, 'Who Made Who', 2)


In [47]:
# 몇 줄 안되는데 길이가 길어지니까, 별칭을 다는것임.

cur.execute("select artist.name, album.title from artist cross join album on artist.id=album.artist_id")

In [48]:
for row in cur.fetchall():
    print(row)

('Led Zepplin', 'IV')
('AC/DC', 'Who Made Who')


In [49]:
# album이랑 track을 조인하고싶어.
# Inner Join(교집합).

cur.execute("select * from album join track on album.id=track.album_id")

for row in cur.fetchall():
    print(row)

(1, 'IV', 1, 1, 'Black Dog', None, None, None, 1, 1)
(1, 'IV', 1, 2, 'Stairway', None, None, None, 1, 2)
(2, 'Who Made Who', 2, 3, 'About to Rock', None, None, None, 2, 1)
(2, 'Who Made Who', 2, 4, 'Who Made Who', None, None, None, 2, 2)


In [50]:
#tack_id == album_id, track_id == genre_id, album_id == artist_id 조인.

cur.execute("select * from track join album join genre join artist \
            on album.id = track.album_id and \
            track.genre_id = genre.id and \
            album.artist_id = artist.id")

for row in cur.fetchall():
    print(row)

(1, 'Black Dog', None, None, None, 1, 1, 1, 'IV', 1, 1, 'Rock', 1, 'Led Zepplin')
(2, 'Stairway', None, None, None, 1, 2, 1, 'IV', 1, 2, 'Metal', 1, 'Led Zepplin')
(3, 'About to Rock', None, None, None, 2, 1, 2, 'Who Made Who', 2, 1, 'Rock', 2, 'AC/DC')
(4, 'Who Made Who', None, None, None, 2, 2, 2, 'Who Made Who', 2, 2, 'Metal', 2, 'AC/DC')


In [51]:
# 명시적으로 알아보기 쉽게 컬럼을 지정해주기.

cur.execute("select track.title, album.title, genre.name, artist.name from track join album join genre join artist \
            on album.id = track.album_id and \
            track.genre_id = genre.id and \
            album.artist_id = artist.id")

for row in cur.fetchall():
    print(row)
    
#  곡명  /  앨범명  /  장르명  /  가수

('Black Dog', 'IV', 'Rock', 'Led Zepplin')
('Stairway', 'IV', 'Metal', 'Led Zepplin')
('About to Rock', 'Who Made Who', 'Rock', 'AC/DC')
('Who Made Who', 'Who Made Who', 'Metal', 'AC/DC')


In [54]:
# 다른곳에 넘겨줄 때 'dump뜬다'고 함.
# memory에있는 dump명령어로 한줄한줄 읽어서 file db에 쓰면됨.


# 쓸려고 열건지('w'), 읽으려고 열건지('r').
# test.dump => Database 파일 확장자로 저장됨. VisualStudio, Atom으로 열릴수있을듯. 글자깨지넹;;;
# dump.sql => SQL 파일 확장자로 저장됨. mySQL 워크벤치, 메모장, text editor로 열림.
# 어떤이름으로 해도 됨.

with open('test.dump', 'w') as f:
    for line in conn.iterdump():
        f.write('%s\n' %line)

In [57]:
data = None
with open('test.dump', 'r') as f:
    data = f.read()

In [74]:
# data저장된 dump.db파일 생성됨.

conn = sqlite3.connect("test.db")
cur = conn.cursor()
cur.executescript(data)

In [ ]:
# track에 있는애랑 left join할것.

conn.execute("insert into track (title) values ('아무거나')")

In [76]:
conn.execute("select * from track")
print(cur.fetchall())

[]


In [77]:
for row in cur.fetchall():
    print(row)

In [78]:
cur.execute("select * from album left join track on album.id=track.album_id")

for row in cur.fetchall():
    print(row)

(1, 'IV', 1, 1, 'Black Dog', None, None, None, 1, 1)
(1, 'IV', 1, 2, 'Stairway', None, None, None, 1, 2)
(2, 'Who Made Who', 2, 3, 'About to Rock', None, None, None, 2, 1)
(2, 'Who Made Who', 2, 4, 'Who Made Who', None, None, None, 2, 2)
